**Import thư viện cần thiết**

In [53]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from data_processing import *

**Xử lí dữ liệu**

In [19]:
# Đọc file dữ liệu mùa giải hiện tại và chuyển vào dataframe
df_saved_name = 'EPL23-24.csv'
df_ml = pd.read_csv(df_saved_name)  # Đọc file CSV vào dataframe df_ml
df_ml


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,NaN,NaN,1.92,1.95
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,19/05/2024,16:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,2.78,-0.75,1.73,2.08,1.78,2.16,1.93,2.21,1.81,2.05
376,E0,19/05/2024,16:00,Liverpool,Wolves,2,0,H,2,0,...,5.17,-2.75,2.07,1.86,2.04,1.85,2.10,1.89,2.04,1.82
377,E0,19/05/2024,16:00,Luton,Fulham,2,4,A,1,2,...,2.69,0.25,2.00,1.93,1.99,1.93,2.02,1.94,1.96,1.91
378,E0,19/05/2024,16:00,Man City,West Ham,3,1,H,2,1,...,4.82,-3.00,2.03,1.90,1.99,1.90,2.05,1.99,1.96,1.91


In [40]:
# Điền giá trị 0 vào các ô bị thiếu
df_ml = df_ml.fillna(0)  # Điền giá trị 0 vào các ô trống trong dataframe df_ml

# Chọn các cột cần giữ lại
columns_to_keep = ['Time', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'Referee', 'HTR']
df_keep = df_ml[columns_to_keep]  # Tạo dataframe df_keep chứa các cột đã chọn

# Chọn các cột cần chuẩn hóa
columns_to_scale = [col for col in df_ml.columns if col not in columns_to_keep]
df_scale = df_ml[columns_to_scale]  # Tạo dataframe df_scale chứa các cột cần chuẩn hóa

# Chuẩn hóa các cột đã chọn
scaled_np = preprocessing.scale(df_scale)  # Chuẩn hóa dữ liệu trong df_scale
scaled_df = pd.DataFrame(scaled_np, columns=df_scale.columns)  # Tạo dataframe mới từ dữ liệu đã chuẩn hóa

# Kết hợp các cột đã giữ lại và các cột đã chuẩn hóa
data = pd.concat([df_keep, scaled_df], axis=1)  # Kết hợp df_keep và scaled_df

# Chọn các cột mong muốn
desired_columns = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
data = data.loc[:, desired_columns]  # Tạo dataframe data chỉ chứa các cột mong muốn
data

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,Burnley,Man City,-1.319824,1.191927,A,-1.503528,0.882414,-1.539688,1.414263,0.073930,-0.912093,-0.038444,0.098819,-1.373769,-1.630966,3.546389,-0.288089
1,Arsenal,Nott'm Forest,0.146647,-0.375313,H,-0.064383,-1.132054,0.542452,-0.963701,0.363552,0.177829,0.583211,-0.590196,0.064336,-0.189647,-0.260469,-0.288089
2,Bournemouth,West Ham,-0.586588,-0.375313,D,-0.224288,0.699281,-0.151594,-0.567374,-0.505314,0.722790,1.204866,-0.245689,-0.654716,1.251671,-0.260469,-0.288089
3,Brighton,Luton,1.613118,-0.375313,H,1.854477,-0.582654,2.277570,-0.567374,0.073930,0.177829,-0.038444,0.787835,0.064336,-0.189647,-0.260469,-0.288089
4,Everton,Fulham,-1.319824,-0.375313,A,0.575237,-0.582654,1.236499,-0.963701,0.363552,-1.457054,1.204866,-0.245689,-1.373769,-0.189647,-0.260469,-0.288089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Crystal Palace,Aston Villa,2.346354,-1.158933,H,-0.064383,-0.765787,1.236499,-0.963701,-0.215692,-0.912093,-1.281755,-0.245689,-0.654716,1.251671,-0.260469,-0.288089
376,Liverpool,Wolves,0.146647,-1.158933,H,3.293622,-1.498321,2.971616,-0.567374,0.942795,-0.094651,1.204866,-0.934704,-0.654716,-0.910306,-0.260469,3.243321
377,Luton,Fulham,0.146647,1.975548,A,-0.064383,0.516147,0.195429,1.017935,1.232417,2.357673,-0.660100,-0.245689,2.221494,1.251671,-0.260469,-0.288089
378,Man City,West Ham,0.879883,-0.375313,H,2.014382,-1.681455,2.277570,-0.963701,-2.243044,0.177829,1.515693,-0.934704,-1.373769,-0.910306,-0.260469,-0.288089


In [41]:
# Đọc file dữ liệu mùa giải trước và chuyển vào dataframe
last_season = 'EPL23-24.csv'
last_season = pd.read_csv(last_season)  # Đọc file CSV vào dataframe last_season

# Điền giá trị 0 vào các ô bị thiếu
last_season = last_season.fillna(0)  # Điền giá trị 0 vào các ô trống trong dataframe last_season

# Tính tỷ lệ thắng trên sân nhà của mỗi đội từ cột 'FTR' (Full Time Result)
home_win = last_season.groupby('HomeTeam')['FTR'].apply(lambda x: (x == 'H').mean())
# Tính tỷ lệ hòa trên sân nhà của mỗi đội từ cột 'FTR'
home_draw = last_season.groupby('HomeTeam')['FTR'].apply(lambda x: (x == 'D').mean())
# Tính tỷ lệ thắng trên sân khách của mỗi đội từ cột 'FTR'
away_win = last_season.groupby('AwayTeam')['FTR'].apply(lambda x: (x == 'A').mean())
# Tính tỷ lệ hòa trên sân khách của mỗi đội từ cột 'FTR'
away_draw = last_season.groupby('AwayTeam')['FTR'].apply(lambda x: (x == 'D').mean())

# Tạo DataFrame mới từ các Series trên
last_season_stats = pd.DataFrame({
    'LSHW': home_win,  # LSHW: Tỷ lệ thắng trên sân nhà mùa trước
    'LSHD': home_draw, # LSHD: Tỷ lệ hòa trên sân nhà mùa trước
    'LSAW': away_win,  # LSAW: Tỷ lệ thắng trên sân khách mùa trước
    'LSAD': away_draw  # LSAD: Tỷ lệ hòa trên sân khách mùa trước
}).reset_index()

# Đổi tên cột 'index' thành 'Team' để dễ dàng merge
last_season_stats.rename(columns={'index': 'Team'}, inplace=True)

last_season_stats

,Team,LSHW,LSHD,LSAW,LSAD
0,Arsenal,0.789474,0.105263,0.684211,0.157895
1,Aston Villa,0.631579,0.210526,0.421053,0.210526
2,Bournemouth,0.368421,0.315789,0.315789,0.157895
3,Brentford,0.263158,0.368421,0.263158,0.105263
4,Brighton,0.421053,0.315789,0.210526,0.315789
5,Burnley,0.105263,0.210526,0.157895,0.263158
6,Chelsea,0.578947,0.210526,0.368421,0.263158
7,Crystal Palace,0.421053,0.210526,0.263158,0.315789
8,Everton,0.421053,0.210526,0.263158,0.263158
9,Fulham,0.473684,0.105263,0.210526,0.315789


In [42]:
# Lấy danh sách các đội từ cột 'HomeTeam' trong dữ liệu
teams = data['HomeTeam'].unique()
# Chuẩn bị tập dữ liệu cho đội khách
prepare_dataset_away = prepare_dataset_away_l5m(data, teams)
# Điền các giá trị NaN bằng 0
prepare_dataset_away = prepare_dataset_away.fillna(0)
# Chuẩn bị tập dữ liệu cho đội nhà
prepare_dataset_home = prepare_dataset_home_l5m(data, teams)
# Điền các giá trị NaN bằng 0
prepare_dataset_home = prepare_dataset_home.fillna(0)

# Chuyển đổi cột 'FTR' thành dạng số (mã hóa)
prepare_dataset_away['FTR'] = prepare_dataset_away['FTR'].astype('category').cat.codes
prepare_dataset_home['FTR'] = prepare_dataset_home['FTR'].astype('category').cat.codes

# Kết hợp hai dataframe lại thành một
merged_df = pd.merge(prepare_dataset_away, prepare_dataset_home, on=['FTR', 'HomeTeam', 'AwayTeam'], suffixes=('_away', '_home'))

merged_df

NameError: name 'prepare_dataset_away_l5m' is not defined

In [25]:
# Chọn các cột cần thiết từ dataframe đã kết hợp
columns_away = ['AC_away', 'AF_away', 'AR_away', 'AS_away', 'AST_away', 'AY_away', 'FTAG_away']
columns_home = ['FTHG_home', 'HC_home', 'HF_home', 'HR_home', 'HS_home', 'HST_home', 'HY_home']
columns_to_group = ['FTR', 'HomeTeam', 'AwayTeam'] + columns_away + columns_home

# Tạo dataframe mới và nhóm theo 'FTR', 'HomeTeam', 'AwayTeam'
new_df = merged_df[columns_to_group].groupby(['FTR', 'HomeTeam', 'AwayTeam']).mean().reset_index()
new_df

,FTR,HomeTeam,AwayTeam,AC_away,AF_away,AR_away,AS_away,AST_away,AY_away,FTAG_away,FTHG_home,HC_home,HF_home,HR_home,HS_home,HST_home,HY_home
0,0,Arsenal,Aston Villa,0.477778,-0.040155,0.065052,0.076627,0.264914,0.458946,0.408307,1.026530e+00,-0.038444,0.073930,-0.260469,0.223446,0.264834,0.064336
1,0,Arsenal,West Ham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.865885e-01,-0.162775,-0.447389,0.500902,0.287408,0.264834,-0.798527
2,0,Aston Villa,Man United,-0.314590,-0.121899,0.065052,-0.179760,-0.250312,0.242748,-0.140227,-2.932942e-01,0.085887,0.537325,-0.260469,-0.064383,-0.012785,1.071010
3,0,Aston Villa,Newcastle,-0.211238,-0.012907,-0.288089,-0.198073,-0.091781,0.242748,0.094859,-2.220446e-17,-0.287107,-0.041919,-0.260469,-0.064383,-0.012785,0.783389
4,0,Aston Villa,Tottenham,0.271073,-0.067403,0.418193,0.113254,0.106383,0.026551,0.565031,1.466471e-01,0.583211,-0.215692,-0.260469,0.159484,0.403643,0.783389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2,Wolves,Fulham,0.029918,-0.094651,0.065052,-0.253013,0.066750,-0.045515,-0.453675,-6.661338e-17,0.023721,-0.273616,-0.260469,-0.160326,-0.082190,0.208147
376,2,Wolves,Luton,-0.245689,-0.012907,-0.288089,-0.619280,-0.448476,-0.333779,0.094859,-7.332356e-01,-0.597934,0.016005,-0.260469,-0.704003,-0.568023,0.927199
377,2,Wolves,Man City,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
378,2,Wolves,Sheffield United,-0.796901,0.341318,0.065052,-0.967234,-0.924068,0.242748,-0.532037,-8.881784e-17,-0.038444,-0.215692,-0.260469,-0.256269,0.056620,-0.079474


In [17]:

# Sao chép dataframe hiện tại
current_df_copy = new_df.copy()

# Merge `current_df_copy` với `last_season_stats` để lấy thông số cho đội nhà
current_df_copy = current_df_copy.merge(last_season_stats[['Team', 'LSHW', 'LSHD']], left_on='HomeTeam', right_on='Team', how='left')
current_df_copy.rename(columns={'LSHW': 'LSHW_home', 'LSHD': 'LSHD_home'}, inplace=True)

# Merge `current_df_copy` với `last_season_stats` một lần nữa để lấy thông số cho đội khách
current_df_copy = current_df_copy.merge(last_season_stats[['Team', 'LSAW', 'LSAD']], left_on='AwayTeam', right_on='Team', how='left', suffixes=('', '_away'))
current_df_copy.rename(columns={'LSAW': 'LSAW_away', 'LSAD': 'LSAD_away'}, inplace=True)

# Loại bỏ các cột 'Team' và 'Team_away' không cần thiết
current_df_copy.drop(['Team', 'Team_away'], axis=1, inplace=True)

# Tạo một bản đồ thay thế cho các đội đặc biệt
special_teams_map = {
    'Burnley': 'Leicester',
    'Sheffield United': 'Leeds',
    'Luton': 'Southampton'
}

# Thay thế thông số của các đội đặc biệt bằng thông số của đội tương ứng
for team, replace_with in special_teams_map.items():
    # Lấy thông số của đội thay thế
    replace_stats = last_season_stats.loc[last_season_stats['Team'] == replace_with, ['LSHW', 'LSHD', 'LSAW', 'LSAD']].iloc[0]
    
    # Cập nhật thông số cho đội cần thay thế trong DataFrame hiện tại
    current_df_copy.loc[current_df_copy['HomeTeam'] == team, ['LSHW_home', 'LSHD_home']] = replace_stats[['LSHW', 'LSHD']].values
    current_df_copy.loc[current_df_copy['AwayTeam'] == team, ['LSAW_away', 'LSAD_away']] = replace_stats[['LSAW', 'LSAD']].values

# Mã hóa các biến phân loại
current_df_copy['HomeTeam'] = current_df_copy['HomeTeam'].astype('category').cat.codes
current_df_copy['AwayTeam'] = current_df_copy['AwayTeam'].astype('category').cat.codes

# Định nghĩa các biến đặc trưng và biến mục tiêu
X = current_df_copy.drop('FTR', axis=1)
y = current_df_copy['FTR']

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


IndexError: single positional indexer is out-of-bounds

In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np

In [4]:
param_grid = {
    'n_estimators': [100, 200],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize the classifier
rf = RandomForestClassifier(random_state=42)

# Initialize the GridSearchCV with error handling
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, error_score=np.nan)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Predict with the best model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Print the accuracy score
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best parameters found:  {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.58      0.61        33
           1       0.00      0.00      0.00         7
           2       0.64      0.75      0.69        36

    accuracy                           0.61        76
   macro avg       0.43      0.44      0.44        76
weighted avg       0.59      0.61      0.59        76

Confusion Matrix:
[[19  2 12]
 [ 4  0  3]
 [ 6  3 27]]
Accuracy Score:
0.6052631578947368


In [5]:
# Import necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1,3, 5, 7, 9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]
}

# Initialize the KNN classifier
knn = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Predict with the best model
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix 
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Print the accuracy score
print("Accuracy Score:") 
print(accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 252 candidates, totalling 1260 fits
Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 41, 'p': 1, 'weights': 'uniform'}
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.21      0.30        33
           1       0.00      0.00      0.00         7
           2       0.51      0.86      0.64        36

    accuracy                           0.50        76
   macro avg       0.34      0.36      0.31        76
weighted avg       0.46      0.50      0.43        76

Confusion Matrix:
[[ 7  0 26]
 [ 3  0  4]
 [ 4  1 31]]
Accuracy Score:
0.5


In [6]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np

# Define the parameter grid for GridSearchCV specific to Decision Tree
# Define the parameter grid for GridSearchCV specific to Decision Tree
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]  # Removed 'auto' from the options
}

# Initialize the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Predict with the best model
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Print the accuracy score
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
Best parameters found:  {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2}
Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.55      0.60        33
           1       0.00      0.00      0.00         7
           2       0.66      0.75      0.70        36

    accuracy                           0.59        76
   macro avg       0.44      0.43      0.43        76
weighted avg       0.60      0.59      0.59        76

Confusion Matrix:
[[18  5 10]
 [ 3  0  4]
 [ 6  3 27]]
Accuracy Score:
0.5921052631578947


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Define the parameter grid for GridSearchCV
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=gnb, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Predict with the best model
best_gnb = grid_search.best_estimator_
y_pred = best_gnb.predict(X_test)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Print the accuracy score
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters found:  {'var_smoothing': 0.02848035868435802}
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.30      0.40        33
           1       0.00      0.00      0.00         7
           2       0.53      0.86      0.65        36

    accuracy                           0.54        76
   macro avg       0.37      0.39      0.35        76
weighted avg       0.50      0.54      0.48        76

Confusion Matrix:
[[10  0 23]
 [ 2  0  5]
 [ 5  0 31]]
Accuracy Score:
0.5394736842105263


c:\Users\huyho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\huyho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\huyho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Định nghĩa tham số grid cho GridSearchCV
param_grid = {
    'C': [0.1, 1, 10], 
    'gamma': [0.0001, 0.001, 0.01, 0.1],
    'kernel': ['rbf', 'linear']
}

# Khởi tạo mô hình SVM 
svm = SVC()

# Khởi tạo GridSearchCV với scoring phù hợp cho bài toán phân loại đa lớp
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Huấn luyện mô hình với GridSearchCV
grid_search.fit(X_train, y_train)

# Lấy siêu tham số tốt nhất
best_params = grid_search.best_params_
print("Siêu tham số tốt nhất: ", best_params)

# Đánh giá mô hình với siêu tham số tốt nhất
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test)

# In ra báo cáo phân loại
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))

# In ra ma trận nhầm lẫn
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred))

# In ra độ chính xác
print("Độ chính xác:")
print(accuracy_score(y_test, y_pred))

Siêu tham số tốt nhất:  {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
Báo cáo phân loại:
              precision    recall  f1-score   support

           0       0.62      0.45      0.53        33
           1       0.00      0.00      0.00         7
           2       0.59      0.83      0.69        36

    accuracy                           0.59        76
   macro avg       0.40      0.43      0.41        76
weighted avg       0.55      0.59      0.56        76

Ma trận nhầm lẫn:
[[15  1 17]
 [ 3  0  4]
 [ 6  0 30]]
Độ chính xác:
0.5921052631578947
